# Statistical analysis comparing PEA test set (MAXMAG clipped) and flight data

This compares the ASVT data set with MAXMAG clipped to flight data since
2019-July-01 (approx start time of MAXMAG clipping in flight products).

Summary: There is reasonable agreement for the 60 and 160 arcsec box sizes.

Reference page:
https://occweb.cfa.harvard.edu/twiki/bin/view/Aspect/PeaAcqModelCalMaxMag11p2Testing

In [1]:
import numpy as np
from pathlib import Path
import os
from collections import Counter
import itertools

from scipy.stats import binom
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib import patches
import tables
from cxotime import CxoTime
import agasc

from utils_asvt import flatten_pea_test_data, read_twiki_csv

%matplotlib inline

In [2]:
SKA = Path(os.environ['SKA'])
HALFWIDTH = 60  # Do analysis for one search box size

In [3]:
topic = 'PeaAcqModelCalDec2022Testing/'
name = 'pea_analysis_2022_336_AcqProbModel_calibration_test_results.csv'
dat8 = read_twiki_csv(topic + name)

In [4]:
dat = flatten_pea_test_data(dat8)

In [5]:
', '.join(sorted(dat.colnames))

'bg_col_offset, bg_row_offset, ccd_temp, hot_px_brightness, image_col, image_fcn, image_row, int_time, max_num_search_hits, maxmag, maxmag_cap, rand_ind, ref_obsid, ref_time, sample_time, scale_4c, search_box_hw, search_success, slot, slot_box_hw, star_col, star_mag, star_maxmag, star_row, star_sz, test_id, y_ang_offset, z_ang_offset'

In [6]:
from proseco.acq import get_maxmag

In [7]:
Counter(dat['ccd_temp'])

Counter({-15: 1728,
         -13: 1728,
         -11: 1728,
         -9: 1728,
         -8: 1728,
         -7: 1728,
         -6: 1728,
         -5: 1728,
         -4: 1728,
         -3: 1728,
         -2: 1728,
         0: 1728,
         2: 1728})

In [8]:
Counter(dat['star_mag'])

Counter({8.0: 2808,
         9.0: 2808,
         9.5: 2808,
         9.75: 2808,
         10.0: 2808,
         10.25: 2808,
         10.5: 2808,
         10.75: 2808})

In [39]:
get_maxmag(60, 0)

10.9

In [38]:
ok = (dat['search_box_hw'] == 60) & (dat['ccd_temp'] == 0)
set(dat['star_maxmag'][ok])

{10.9, 11.0, 11.25}

In [11]:
set(dat['maxmag'])

{99.9}

In [29]:
from proseco.core import StarsTable
from proseco import characteristics as ACA, get_aca_catalog
from proseco.tests.test_common import DARK40, STD_INFO, mod_std_info
from proseco.acq import get_maxmag


In [35]:
stars = StarsTable.empty()
mag0 = ACA.max_maxmag - 1.5  # nominal star mag when clipping occurs (11.2 - 1.5 = 9.7)
mags_acq = np.array([-1.5, -1, -0.5, -0.01, 0.01, 0.2, 0.3, 1.1]) + mag0
mags_guide = np.array([-0.5, -0.01, 0.01, 0.2, 0.3]) + mag0
stars.add_fake_constellation(mag=mags_acq, n_stars=8, size=2000)
stars.add_fake_constellation(mag=mags_guide, n_stars=5, size=1000)
aca = get_aca_catalog(stars=stars, dark=DARK40, raise_exc=True,
                        exclude_ids_guide=np.arange(100, 108),
                        include_ids_acq=np.arange(100, 108),
                        exclude_ids_acq=np.arange(108, 113),
                        **mod_std_info(t_ccd=-1))


/Users/aldcroft/miniconda3/envs/ska3/lib/python3.8/site-packages/proseco/acq.py:70: UserWarning: Clipping t_ccd=4.5 to 0.0 for interpolating MAXMAGs table
  warnings.warn(f"Clipping {t_ccd=} to 0.0 for interpolating MAXMAGs table")
/Users/aldcroft/miniconda3/envs/ska3/lib/python3.8/site-packages/chandra_aca/star_probs.py:288: UserWarning: 
Model grid-floor-2020-02 computed between t_ccd <= -16.0 <= -1.0, clipping input t_ccd(s) outside that range.
  warnings.warn('\nModel {} computed between {} <= {} <= {}, '
/Users/aldcroft/miniconda3/envs/ska3/lib/python3.8/site-packages/proseco/acq.py:70: UserWarning: Clipping t_ccd=4.5 to 0.0 for interpolating MAXMAGs table
  warnings.warn(f"Clipping {t_ccd=} to 0.0 for interpolating MAXMAGs table")


In [36]:
aca

slot,idx,id,type,sz,p_acq,mag,maxmag,yang,zang,dim,res,halfw
int64,int64,int64,str3,str3,float64,float64,float64,float64,float64,int64,int64,int64
0,1,1,FID,8x8,0.000,7.00,8.00,922.59,-1737.89,1,1,25
1,2,5,FID,8x8,0.000,7.00,8.00,-1826.28,160.17,1,1,25
2,3,6,FID,8x8,0.000,7.00,8.00,388.59,803.75,1,1,25
0,4,100,ACQ,8x8,0.908,8.20,9.70,2000.00,0.00,28,1,160
1,5,101,ACQ,8x8,0.825,8.70,10.20,0.00,2000.00,28,1,160
2,6,102,ACQ,8x8,0.758,9.20,10.66,-2000.00,0.00,12,1,80
3,7,103,ACQ,8x8,0.301,9.69,10.90,0.00,-2000.00,8,1,60
4,8,104,ACQ,8x8,0.280,9.71,10.90,1000.00,1000.00,8,1,60
5,9,105,ACQ,8x8,0.069,9.90,10.43,1000.00,-1000.00,24,1,140


In [32]:
aca.t_ccd_eff_acq

4.5

In [33]:
get_maxmag(80, 4.5)

/Users/aldcroft/miniconda3/envs/ska3/lib/python3.8/site-packages/proseco/acq.py:70: UserWarning: Clipping t_ccd=4.5 to 0.0 for interpolating MAXMAGs table
  warnings.warn(f"Clipping {t_ccd=} to 0.0 for interpolating MAXMAGs table")


10.66